In [1]:
print("all ok")

all ok


In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

In [3]:
BASE_DIR =Path().resolve()

In [4]:
BASE_DIR

WindowsPath('C:/Users/giris/semantic-image-search/semantic_image_search/notebooks')

In [5]:
images_root = BASE_DIR/"images"

In [6]:
images_root

WindowsPath('C:/Users/giris/semantic-image-search/semantic_image_search/notebooks/images')

In [7]:
Model_id = "ViT-B-32__laion2b-s34b-b79k"

In [8]:
Model_id = "openai/clip-vit-base-patch32"

In [22]:
load_dotenv()

True

In [10]:
from langchain_experimental.open_clip import OpenCLIPEmbeddings

In [11]:
embedder = OpenCLIPEmbeddings(
    model_name="ViT-B-32",
    checkpoint="laion2b_s34b_b79k", 
    device="cpu"
)


c:\Users\giris\semantic-image-search\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
str(images_root/"animal"/"cat.jpg")

'C:\\Users\\giris\\semantic-image-search\\semantic_image_search\\notebooks\\images\\animal\\cat.jpg'

In [14]:
img_embedding = embedder.embed_image([str(images_root/"animal"/"cat.jpeg")])

In [15]:
len(img_embedding)

1

In [16]:
len(img_embedding[0])

512

In [17]:
img_embedding

[[0.04853694513440132,
  -0.0014377229381352663,
  -0.03448808938264847,
  -0.08087947964668274,
  0.026776470243930817,
  0.07083483040332794,
  -0.008278947323560715,
  -0.048146240413188934,
  0.05869951471686363,
  0.01400085724890232,
  0.01861771196126938,
  -0.026211250573396683,
  -0.024436134845018387,
  -0.05617030709981918,
  -0.013328208588063717,
  0.025808153674006462,
  -0.12121991813182831,
  -0.02116970717906952,
  -0.010362028144299984,
  0.0012653496814891696,
  0.015890400856733322,
  -0.010824950411915779,
  -0.047347478568553925,
  -0.01107318326830864,
  -0.0681760162115097,
  -0.025547342374920845,
  -0.05428933724761009,
  0.01654183305799961,
  -0.02675372175872326,
  -0.01631147414445877,
  -0.02890767715871334,
  -0.010116596706211567,
  -0.05007088929414749,
  0.03665084019303322,
  -0.051859982311725616,
  -0.001090390607714653,
  0.003514895448461175,
  0.08354642987251282,
  -0.07093144953250885,
  -0.06484895944595337,
  0.062359295785427094,
  0.006774

In [18]:
img_embedding[0]

[0.04853694513440132,
 -0.0014377229381352663,
 -0.03448808938264847,
 -0.08087947964668274,
 0.026776470243930817,
 0.07083483040332794,
 -0.008278947323560715,
 -0.048146240413188934,
 0.05869951471686363,
 0.01400085724890232,
 0.01861771196126938,
 -0.026211250573396683,
 -0.024436134845018387,
 -0.05617030709981918,
 -0.013328208588063717,
 0.025808153674006462,
 -0.12121991813182831,
 -0.02116970717906952,
 -0.010362028144299984,
 0.0012653496814891696,
 0.015890400856733322,
 -0.010824950411915779,
 -0.047347478568553925,
 -0.01107318326830864,
 -0.0681760162115097,
 -0.025547342374920845,
 -0.05428933724761009,
 0.01654183305799961,
 -0.02675372175872326,
 -0.01631147414445877,
 -0.02890767715871334,
 -0.010116596706211567,
 -0.05007088929414749,
 0.03665084019303322,
 -0.051859982311725616,
 -0.001090390607714653,
 0.003514895448461175,
 0.08354642987251282,
 -0.07093144953250885,
 -0.06484895944595337,
 0.062359295785427094,
 0.006774506065994501,
 0.06565640866756439,
 -0.01

In [25]:
url = os.getenv("API_ENDPOINT")

In [26]:
api_key = os.getenv("QDRANT_API_KEY")

In [27]:
from qdrant_client import QdrantClient

In [32]:
qdrant_client = QdrantClient(url=url, api_key=api_key)

In [44]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='semantic-image-search')])

In [45]:
collections = qdrant_client.get_collections().collections

In [46]:
collections

[CollectionDescription(name='semantic-image-search')]

In [36]:
COLLECTION_NAME = "semantic-image-search"
VECTOR_SIZE = 512

In [38]:
from qdrant_client.http import models

In [42]:
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config = models.VectorParams(size=VECTOR_SIZE,distance=models.Distance.COSINE)
)

True

In [47]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='semantic-image-search')])

In [48]:
collections = qdrant_client.get_collections().collections
existing_names = {c.name for c in collections}

In [49]:
existing_names

{'semantic-image-search'}

In [50]:
if COLLECTION_NAME not in existing_names:
    print(f"Creating collection: {COLLECTION_NAME}")
    qdrant_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=VECTOR_SIZE,
            distance=models.Distance.COSINE,
        ),
    )
else:
    print(f"Collection already exists: {COLLECTION_NAME} (reusing)")

Collection already exists: semantic-image-search (reusing)


In [43]:
def index_image(image_path, category=None):
    pass

In [51]:
def index_folder(root_folder):
    pass